In [1]:
#pip install simpledbf 

In [3]:
import pandas as pd
import os
import simpledbf
from simpledbf import Dbf5

# Data treatment Production
### Coal, Wind, Solar and Natural Gas

In [4]:
df_energy = pd.read_csv("Data/EnergyProduction.csv", header = 0)

def convert_float(string): #problème aperçu quand en train de filtrer pour les renouvelables - il y a d'ans où on a aucune information
    if string == "Not Available":
        return(0)
    else:
        return(float(string))
    
df_energy['Year'] = df_energy['YYYYMM'].apply(lambda x: int(str(x)[:4]))
df_energy['Month'] = df_energy['YYYYMM'].apply(lambda x: int(str(x)[4:6]))
df_energy['Unit'] = 'Gigawatthours'
df_energy['Value'] = df_energy['Value'].apply(lambda x: convert_float(x)/1000)

In [5]:
# Data treatment Coal, Wind, Solar and Natural Gas

energy_index = {'coal':1,'gas':3, 'solar':11, 'wind':12}

def regression(annual_value,year):
    return(annual_value/12)

def df_to_annual(df):
    df = df.where(df['Month']==13).dropna()
    df.drop(['YYYYMM'], axis = 1, inplace = True)
    return(df)

def df_to_month(df):
    for year in range(1949,1973,1):
        for month in range(1,13,1):
            i = (year - 1949)*12 + month - 1
            if month<10:
                yyyymm = int(str(year) + '0' + str(month))
            else:
                yyyymm = int(str(year) + str(month))
            value = df.loc[df['Year']==year]['Value'].values[0]
            new_row = pd.DataFrame({'YYYYMM': yyyymm, 'Year': year, 'Month': month, 'Production': regression(value,year), 'Unit':'Gigawatthours'}, index = [i])
            df = df.append(new_row, ignore_index = False)
    return(df)

for energy in energy_index:
    df = df_energy.where(df_energy['Column_Order'] == energy_index[energy]).dropna()
    df.drop(['Column_Order','Description','MSN'], axis = 1, inplace = True)
    df_annual = df_to_annual(df)
    df_tool = df_to_month(df)
    df_month = df_tool.where(df_tool['Month']!=13).dropna()
    file_annual = 'treated_data/' + energy + '_treated.csv'
    file_month = 'treated_data/' + energy + '_month_treated.csv'
    df_annual.to_csv(file_annual, sep=';', index=True)
    df_month.to_csv(file_month, sep=';', index=True)


# Data treatment Capacity

### Création d'un dictionnaire pour qu'on automatise la lecture et création du dataframe

In [7]:
year_infos = {}
for folder in os.listdir('Data/Capacity/'):
    if folder[:6] != 'eia860': #il y a un documents qui je ne sais pas d'où il vient, mais qui ne nous intêret pas
        continue
    year = int(folder[-4:])
    for file in os.listdir(f'Data/Capacity/{folder}'):
        
        if year < 2004: #objectif = 2001 ->2003. On doit les traîter separemment puisqu'ils sont un fichier dbf
            if file[:3].upper() == 'GEN' and file[-3:]=='dbf': #c'est le fichier desire, mais il est encore en format dbf
                    old_path = 'Data/Capacity/'+folder+'/'+file
                    dbf = Dbf5(old_path)
                    new_path = 'Data/Capacity/'+folder+'/'+file[:-3]+'csv'
                    dbf.to_csv(new_path)
                    path = new_path
                    energy_source = 'ENSOURCE1'
                    capacity = 'NAMEPLATE'
                    year_infos[year] = [path,energy_source,capacity]

        
        elif year<= 2012: #il y a eu un changement dans comment les fichiers s'appelent
            if file[:3].upper() == 'GEN':
                if file[-3:] not in ['xls','lsx']:
                    continue # je vais traiter les fichiers .dbf (2001-2003) après
                path = 'Data/Capacity/'+folder+'/'+file
                energy_source = 'ENERGY_SOURCE_1'
                capacity = 'NAMEPLATE'
                if year == 2012:
                    energy_source = 'Energy Source 1'
                    capacity = 'Nameplate Capacity (MW)'
                
                year_infos[year] = [path,energy_source,capacity]
        else:
            if file[:4] == '3_1_':
                path = 'Data/Capacity/'+folder+'/'+file
                energy_source = 'Energy Source 1'
                capacity = 'Nameplate Capacity (MW)'
                year_infos[year] = [path,energy_source,capacity]

##### Création d'une list avec les codes d'interêt et d'un dictionnaire 
Le but c'est de s'en servir pour créer une colomne ('Energy Source') à partir de groupby(codes), puisque les fichiers plus ancians n'ont pas une colomne 'Technology'.

In [8]:
path_2020, energy_source_2020, capacity_2020= year_infos[2020][0],year_infos[2020][1], year_infos[2020][2]
df_2020 = pd.read_excel(path_2020,header = 1)

energies = ['Onshore Wind Turbine','Natural Gas Steam Turbine', 'Conventional Steam Coal', 'Solar Photovoltaic']
technologies = {}
codes = []

for technology in df_2020['Technology'].unique():
    if technology in energies: 
        energy_source = df_2020['Energy Source 1'].where(df_2020['Technology']==technology).dropna().unique()
        print(technology,energy_source)
        technologies[technology] = energy_source
        codes.append(energy_source)

#print(technologies)
#print(codes)

Onshore Wind Turbine ['WND']
Natural Gas Steam Turbine ['NG']
Conventional Steam Coal ['BIT' 'LIG' 'SUB' 'RC' 'WC']
Solar Photovoltaic ['SUN']


In [9]:
technologies

{'Onshore Wind Turbine': array(['WND'], dtype=object),
 'Natural Gas Steam Turbine': array(['NG'], dtype=object),
 'Conventional Steam Coal': array(['BIT', 'LIG', 'SUB', 'RC', 'WC'], dtype=object),
 'Solar Photovoltaic': array(['SUN'], dtype=object)}

In [49]:
codes = ['WND','NG','BIT','LIG','SUB','RC','WC','SUN'] #fait en regardant l'output de l'itération dessus

### Lecture automatisé de chaque année

##### Création d'un fonction pour traîter le type de fichier 2001-2003
Il a plusieurs en-tetes (headers) et quelques lignes de "NAMEPLATE" ne sont pas en format float

In [50]:
def delete_other_headers(df): #dans les dfs plus agés (2001-03), il y avait plus d'un entete (header), et les numéros parfois n'étaient pas tous en format float
    df_interest = pd.DataFrame()
    df_interest['ENSOURCE1'],df_interest['NAMEPLATE'] = df['ENSOURCE1'],df['NAMEPLATE']
    #on doit effacer les lignes qui sont entetes, c-a-d: on trouvera celles où le valuer dans 'NAMEPLATE' 
    #est aussi 'NAMEPLATE'
    index = 0
    indexes=[]
    for line in df['NAMEPLATE']:
        if line == "NAMEPLATE":
            indexes.append(index)
        index+=1
    df_interest = df_interest.drop(indexes,axis=0)
    df_interest['NAMEPLATE'] = df_interest['NAMEPLATE'].apply(lambda x: float(x))
    #On a pu le faire directment puisque toutes les lignes étaient en format '1231.2', s'il avait quelqu'une differente,
    #il aurait jeté une exception
    return(df_interest)

In [51]:
energies = ['Onshore Wind Turbine','Natural Gas Steam Turbine', 'Conventional Steam Coal', 'Solar Photovoltaic']

df_capacity = pd.DataFrame()

for year in year_infos:
    path, energy_source, capacity = year_infos[year][0],year_infos[year][1], year_infos[year][2]

    if year <=2003:
        df_tool = pd.read_csv(path,header = 0)
        df_year_raw = delete_other_headers(df_tool)

    elif year<=2010:
        df_year_raw = pd.read_excel(path,header = 0)
    
    else: 
        df_year_raw = pd.read_excel(path,header = 1)
    
    df_year = df_year_raw.groupby(energy_source, as_index = True).sum()[capacity]
    
    for i in range(len(df_year)):
        code = df_year.index[i]
        if code in codes:
            value = df_year[i]
            new_row = pd.DataFrame({'Year': year,'Energy Source': code, 'Capacity': value}, index = [i])
            df_capacity = df_capacity.append(new_row, ignore_index = False)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,5,6,7,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,5,6,7,17,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Column "Technology"

In [52]:
codes_technologies = {'WND':'Onshore Wind Turbine',
                      'NG':'Natural Gas Steam Turbine',
                      'SUN':'Solar Photovoltaic'}
for code in technologies['Conventional Steam Coal']:
    codes_technologies[code]='Conventional Steam Coal'

In [53]:
df_capacity['Technology'] = df_capacity['Energy Source'].apply(lambda x: codes_technologies[x])

#### Création d'un csv sans l'especification du type d'energie

In [55]:
df_tool = df_capacity.groupby(['Year','Technology']).sum()
df_capacity_general = pd.DataFrame()

for i in range(len(df_tool)): # ici on utilise le range normal, 0 -> 2021-1973, puisque les ans presents dans df_renewable_month sont les indexes
        year = df_tool.iloc[i].name[0]
        technology = df_tool.iloc[i].name[1]
        capacity = df_tool.iloc[i]['Capacity']
        new_row = pd.DataFrame({'Year': year, 'Technology': technology, 'Capacity': capacity}, index = [i])
        df_capacity_general=df_capacity_general.append(new_row, ignore_index = False)

#### Saving to csv

In [57]:
df_capacity.to_csv('treated_data/capacity.csv', sep=';', index=True)
df_capacity_general.to_csv('treated_data/capacity_general.csv')

In [30]:
df_capacity = pd.read_csv('treated_data/Capacity/capacity.csv',sep=';').drop('Unnamed: 0', axis=1)

# Merge Production & Capacity on Year 
#### Il manque encore (1) comprendre quelle est l'unité de capacity (2) capacity factor (je ne peux pas trouver
#### quelque chose qui dit la durée max dans les excels.

In [20]:
energy_infos = {'solar':['treated_data/Production/solar_treated.csv','Solar Photovoltaic'],
                      'wind':['treated_data/Production/wind_treated.csv','Onshore Wind Turbine'],
                      'coal':['treated_data/Production/coal_treated.csv','Conventional Steam Coal'],
                     'gas':['treated_data/Production/gas_treated.csv','Natural Gas Steam Turbine']}

In [39]:
for energy in energy_infos:
    df_production = pd.read_csv(energy_infos[energy][0],sep = ';').drop('Unnamed: 0', axis=1)
    df_production = df_production.where(df_production['Year']>2000).dropna()
    df_capacity = pd.read_csv('treated_data/Capacity/capacity_general.csv').drop('Unnamed: 0', axis=1)
    df_capacity = df_capacity.where(df_capacity['Technology']==energy_infos[energy][1]).dropna()
    df_energy = pd.merge (df_production, df_capacity, on= 'Year')
    df_energy['Ratio'] = df_energy['Value']/df_energy['Capacity']
    path = 'treated_data/Energies/' + energy + '_treated.csv'
    df_energy.to_csv(path, sep = ';')